In [1]:
import sys
import os

# append the path of the parent directory
sys.path.append("../src")
os.path.abspath(os.getcwd())

'/home/michel/Documents/BPO_Batching/notebooks'

In [2]:
import collections
import numpy as np
from ilp_policy_non_assign import *
from hungarian_policy import *

In [3]:
class Task:
    def __init__(self, name, task_type):
        self.name = name
        self.task_type = task_type

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

In [4]:
t1 = Task('t1', 't1')
t2 = Task('t2', 't2')
t3 = Task('t3', 't3')

unassigned_tasks = [t1, t2, t3]
available_resources = ['Rb']
working_resources = {'Ra' : (-2, 4)}
resource_pool = {
    't1' : ['Ra', 'Rb', 'Rc'],
    't2' : ['Rb', 'Rc'],
    't3' : ['Ra']
}
trds = {
    (t1, 'Ra') : 7,
    (t1, 'Rb') : 14,
    (t1, 'Rc') : 8,
    (t2, 'Rb') : 10,
    (t2, 'Rc') : 15,
    (t3, 'Ra') : 5
}
task_durations = collections.defaultdict(list)
for (t, r), trd in trds.items():
    task_durations[t].append(trd)
mean_durations = {k: np.mean(v) for k, v in task_durations.items()}

In [5]:
print(task_durations)
print(mean_durations)

defaultdict(<class 'list'>, {t1: [7, 14, 8], t2: [10, 15], t3: [5]})
{t1: 9.666666666666666, t2: 12.5, t3: 5.0}


In [6]:
print(unassigned_tasks, available_resources, resource_pool, trds, {}, {}, mean_durations, working_resources, 0)

[t1, t2, t3] ['Rb'] {'t1': ['Ra', 'Rb', 'Rc'], 't2': ['Rb', 'Rc'], 't3': ['Ra']} {(t1, 'Ra'): 7, (t1, 'Rb'): 14, (t1, 'Rc'): 8, (t2, 'Rb'): 10, (t2, 'Rc'): 15, (t3, 'Ra'): 5} {} {} {t1: 9.666666666666666, t2: 12.5, t3: 5.0} {'Ra': (-2, 4)} 0


In [7]:
def do_allocation(policy, unassigned_tasks, available_resources, resource_pool, trds, mean_durations, working_resources, time):
    allocation = policy.allocate(unassigned_tasks, available_resources, resource_pool, trds, {}, {}, mean_durations, working_resources, time)
    return allocation

def add_working_resources(time, trds, allocation, available_resources, working_resources):
    new_available_resources = available_resources.copy()
    new_working_resources = working_resources.copy()
    for t, r in allocation:
        new_working_resources[r] = (time, trds[(t,r)])
        new_available_resources.remove(r)
    return new_available_resources, new_working_resources

def remove_working_resources(time, available_resources, working_resources):
    new_available_resources = available_resources.copy()
    new_working_resources = working_resources.copy()
    for r, ts in working_resources.items():
        if sum(ts) <= time:
            del new_working_resources[r]
            new_available_resources.append(r)
    return new_available_resources, new_working_resources

def remove_unassigned_tasks(allocation, unassigned_tasks):
    new_unassigned_tasks = unassigned_tasks.copy()
    for t,r in allocation:
        new_unassigned_tasks.remove(t)
    return new_unassigned_tasks

def remove_trds(allocation, trds):
    new_trds = trds.copy()
    for t,r in allocation:
        for tt,rr in trds.keys():
            if t==tt:
                del new_trds[(tt,rr)]
    return new_trds

In [8]:
working_resources.values()

dict_values([(-2, 4)])

In [16]:
next_event = lambda working_resources : min(map(sum, working_resources.values())) if working_resources else np.inf


def do_simulation(policy, time, unassigned_tasks, available_resources, working_resources, trds, mean_durations, resource_pool):
    log = ''
    unassigned_tasks = unassigned_tasks.copy()
    available_resources = available_resources.copy()
    working_resources = working_resources.copy()
    trds = trds.copy()
    
    while len(working_resources):
        log += str(time)+'\n'
        available_resources, working_resources = remove_working_resources(time, available_resources, working_resources)
        allocation = do_allocation(policy, unassigned_tasks, available_resources, resource_pool, trds, mean_durations.copy(), working_resources, time)
        unassigned_tasks = remove_unassigned_tasks(allocation, unassigned_tasks)
        available_resources, working_resources = add_working_resources(time, trds, allocation, available_resources, working_resources)
        trds = remove_trds(allocation, trds)
        log += str(allocation)+'\n'
        log += ', '.join(map(str, [unassigned_tasks, available_resources, trds, working_resources]))+'\n'
        last_time = time
        time = next_event(working_resources)
    log += '---------\n'

    return last_time, log


In [21]:
start_time = 0
last_time = start_time
for i in np.arange(-0.01, 4, 0.001):

    policy = UnrelatedParallelMachinesSchedulingNonAssignPolicy(1, 0, 0, i)
    time, log = do_simulation(policy, start_time, unassigned_tasks, available_resources, working_resources, trds, mean_durations, resource_pool)

    if time != last_time:
        print('Delta:', str(i))
        print(log)
        last_time = time
        print('===========================================')

No solution 0 3 2 (0, 50400)
No solution 0 3 2 (0, 43200)
Delta: -0.01
0
[]
[t1, t2, t3], ['Rb'], {(t1, 'Ra'): 7, (t1, 'Rb'): 14, (t1, 'Rc'): 8, (t2, 'Rb'): 10, (t2, 'Rc'): 15, (t3, 'Ra'): 5}, {'Ra': (-2, 4)}
2
[]
[t1, t2, t3], ['Rb', 'Ra'], {(t1, 'Ra'): 7, (t1, 'Rb'): 14, (t1, 'Rc'): 8, (t2, 'Rb'): 10, (t2, 'Rc'): 15, (t3, 'Ra'): 5}, {}
---------

No solution 0 3 2 (0, 50400)
No solution 0 3 2 (0, 43200)
No solution 0 3 2 (0, 50400)
No solution 0 3 2 (0, 43200)
No solution 0 3 2 (0, 50400)
No solution 0 3 2 (0, 43200)
No solution 0 3 2 (0, 50400)
No solution 0 3 2 (0, 43200)
No solution 0 3 2 (0, 50400)
No solution 0 3 2 (0, 43200)
No solution 0 3 2 (0, 50400)
No solution 0 3 2 (0, 43200)
No solution 0 3 2 (0, 50400)
No solution 0 3 2 (0, 43200)
No solution 0 3 2 (0, 50400)
No solution 0 3 2 (0, 43200)
No solution 0 3 2 (0, 50400)
No solution 0 3 2 (0, 43200)
Delta: 0.36099999999999965
0
[(t2, 'Rb')]
[t1, t3], [], {(t1, 'Ra'): 7, (t1, 'Rb'): 14, (t1, 'Rc'): 8, (t3, 'Ra'): 5}, {'Ra': (

In [11]:
time = 0
for i in np.arange(1, 1.5, 0.1):
    print('Delta:', str(i))
    policy = HungarianMultiObjectivePolicy(1, 0, 0, i)
    do_simulation(policy, time, unassigned_tasks, available_resources, working_resources, trds, mean_durations, resource_pool)
    print('===========================================')

Delta: 1.0
Delta: 1.1
Delta: 1.2000000000000002
Delta: 1.3000000000000003
Delta: 1.4000000000000004
